In [1]:
import numpy as np
import pandas as pd
from patsy import dmatrices

import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

# Feature selection using p values leads to overestimates for p values

In [2]:
df = pd.DataFrame()
n = 500
p = 100
for j in range(p):
    xp = 'x'+str(j)
    df[xp] = np.random.randn(n)
y = np.random.randn(n)
df.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
0,-1.761625,-0.952876,0.167463,-0.363252,2.064327,0.999818,1.345103,-0.681554,0.474441,0.217572,...,-0.096401,0.375151,-0.146902,0.744395,-0.414828,-1.565182,0.375126,0.503835,-0.982638,0.843941
1,-0.040303,-0.724593,0.117904,-0.665537,-0.928679,0.948072,-0.137149,0.677710,-0.120825,0.288771,...,-1.321725,-0.753704,-0.660238,-1.628445,-0.229142,2.200838,0.076438,-1.751045,2.056411,-0.019826
2,0.742602,1.011122,-0.281435,1.279151,0.497981,1.263263,1.234160,0.688636,-0.502067,0.321529,...,0.381719,-0.236890,-0.040671,1.347579,0.725742,-0.056826,-0.962921,-0.282061,-1.938646,0.730166
3,-0.703046,0.192244,-1.721894,1.999569,-0.302392,0.296787,-1.088114,0.715013,-1.019179,-1.290164,...,-0.340932,-2.184356,-1.477577,-1.255982,0.791318,1.049995,0.503556,-0.004232,0.742050,-0.432857
4,-1.198074,-0.115802,-0.888837,-0.870881,-0.603077,2.207998,-0.544623,0.841634,0.115917,0.635422,...,-2.500031,-0.436328,-0.191603,-0.267472,1.610789,0.139477,0.332086,-1.172738,-0.311496,1.553304


In [3]:
variables = df.columns
model = sm.OLS(y,df[variables]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.227
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     1.177
Date:                Fri, 17 Apr 2020   Prob (F-statistic):              0.140
Time:                        16:04:56   Log-Likelihood:                -610.79
No. Observations:                 500   AIC:                             1422.
Df Residuals:                     400   BIC:                             1843.
Df Model:                         100                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x0             0.0557      0.043      1.283      0.200      -0.030       0.141
x1            -0.0151      0.046     -0.326      0.744      -0.106       0.076
x2             0.0113      0.045      0.251      0.802      -0.077       0.100
x3             0.0619      0.043      1.424      0.155      -0.024       0.147
x4             0.0839      0.046      1.825      0.069      -0.006       0.174
x5            -0.0532      0.047     -1.142      0.254      -0.145       0.038
x6             0.0237      0.046      0.515      0.607      -0.067       0.114
x7            -0.0201      0.050     -0.401      0.689      -0.119       0.078
x8            -0.0348      0.047     -0.745      0.457      -0.127       0.057
x9            -0.0236      0.044     -0.534      0.594      -0.111       0.063
x10           -0.0506      0.045     -1.124      0.261      -0.139       0.038
x11            0.0230      0.047      0.491      0.624      -0.069       0.115
x12           -0.0245      0.047     -0.521      0.603      -0.117       0.068
x13            0.0197      0.041      0.480      0.632      -0.061       0.100
x14            0.0183      0.045      0.408      0.684      -0.070       0.107
x15            0.1041      0.048      2.151      0.032       0.009       0.199
x16           -0.0086      0.044     -0.196      0.845      -0.095       0.078
x17           -0.0539      0.045     -1.210      0.227      -0.141       0.034
x18           -0.0676      0.044     -1.534      0.126      -0.154       0.019
x19           -0.0051      0.049     -0.105      0.917      -0.101       0.090
x20            0.0587      0.044      1.322      0.187      -0.029       0.146
x21            0.0109      0.045      0.239      0.811      -0.079       0.100
x22           -0.0560      0.047     -1.200      0.231      -0.148       0.036
x23           -0.0156      0.046     -0.341      0.733      -0.105       0.074
x24            0.0153      0.050      0.309      0.757      -0.082       0.113
x25           -0.0803      0.045     -1.802      0.072      -0.168       0.007
x26            0.0979      0.048      2.049      0.041       0.004       0.192
x27            0.0505      0.044      1.143      0.254      -0.036       0.137
x28            0.0386      0.046      0.830      0.407      -0.053       0.130
x29            0.0973      0.046      2.123      0.034       0.007       0.187
x30            0.0207      0.047      0.442      0.659      -0.071       0.113
x31           -0.0699      0.048     -1.460      0.145      -0.164       0.024
x32            0.0075      0.046      0.163      0.870      -0.083       0.098
x33           -0.0953      0.045     -2.124      0.034      -0.183      -0.007
x34            0.0788      0.049      1.608      0.109      -0.018       0.175
x35           -0.0169      0.048     -0.350      0.727      -0.112       0.078
x3

In [4]:
# uncomment this line the first time you run the cell; comment it out for subsequent runs
stat_sig_vars = variables
stat_sig_vars = stat_sig_vars[model.pvalues <= .05]
model = sm.OLS(y,df[stat_sig_vars]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     3.796
Date:                Fri, 17 Apr 2020   Prob (F-statistic):           5.98e-05
Time:                        16:04:56   Log-Likelihood:                -656.63
No. Observations:                 500   AIC:                             1333.
Df Residuals:                     490   BIC:                             1375.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x15            0.0872      0.044      1.996      0.047       0.001       0.173
x26            0.0748      0.043      1.727      0.085      -0.010       0.160
x29            0.0833      0.042      1.996      0.047       0.001       0.165
x33           -0.0927      0.040     -2.292      0.022      -0.172      -0.013
x37           -0.0738      0.040     -1.860      0.064      -0.152       0.004
x45            0.0826      0.041      2.005      0.046       0.002       0.164
x53            0.0561      0.040      1.392      0.165      -0.023       0.135
x89            0.0945      0.042      2.263      0.024       0.012       0.177
x92           -0.1043      0.041     -2.537      0.011      -0.185      -0.024
x99           -0.0971      0.043     -2.267      0.024      -0.181      -0.013
==============================================================================
Omnibus:                        6.301   Durbin-Watson:                   2.029
Prob(Omnibus):                  0.043   Jarque-Bera (JB):                7.239
Skew:                          -0.159   Prob(JB):                       0.0268
Kurtosis:                       3.496   Cond. No.                         1.21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Solution: held out test set

In [5]:
X = df
y = y

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.5, random_state=1)
print("Training set: ", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)

Training set:  (250, 100) (250,)
Test set: (250, 100) (250,)


## select model using the training set, compute statistics on test set

In [10]:
# model selection using p values
variables = X.columns
model = sm.OLS(y_train,X_train[variables]).fit()
stat_sig_vars = variables
while any(model.pvalues > .05):
    stat_sig_vars = stat_sig_vars[model.pvalues <= .05]
    model = sm.OLS(y_train,X_train[stat_sig_vars]).fit()
# statistics on training set 
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.105
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     7.180
Date:                Fri, 17 Apr 2020   Prob (F-statistic):           1.75e-05
Time:                        16:12:00   Log-Likelihood:                -338.12
No. Observations:                 250   AIC:                             684.2
Df Residuals:                     246   BIC:                             698.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x8            -0.1495      0.060     -2.510      0.013      -0.267      -0.032
x15            0.1325      0.062      2.124      0.035       0.010       0.255
x18           -0.1909      0.057     -3.329      0.001      -0.304      -0.078
x34            0.2029      0.064      3.162      0.002       0.077       0.329
==============================================================================
Omnibus:                        1.116   Durbin-Watson:                   1.789
Prob(Omnibus):                  0.572   Jarque-Bera (JB):                0.822
Skew:                          -0.102   Prob(JB):                        0.663
Kurtosis:                       3.194   Cond. No.                         1.17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# report statistics on test set - notice p values are no longer "statistically significant"!
model = sm.OLS(y_test,X_test[stat_sig_vars]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.5527
Date:                Fri, 17 Apr 2020   Prob (F-statistic):              0.697
Time:                        16:12:01   Log-Likelihood:                -320.40
No. Observations:                 250   AIC:                             648.8
Df Residuals:                     246   BIC:                             662.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x8             0.0260      0.059      0.444      0.657      -0.089       0.141
x15            0.0285      0.061      0.465      0.642      -0.092       0.149
x18            0.0678      0.054      1.260      0.209      -0.038       0.174
x34           -0.0246      0.058     -0.423      0.672      -0.139       0.090
==============================================================================
Omnibus:                        9.822   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               11.550
Skew:                          -0.347   Prob(JB):                      0.00310
Kurtosis:                       3.792   Cond. No.                         1.22
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Now for real data

In [12]:
boston_dataset = datasets.load_boston()
boston_dataset

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

In [13]:
print(boston_dataset.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [14]:
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [15]:
X = boston
y = boston_dataset.target

In [16]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.5, random_state=0)

In [17]:
# model selection using p values on training set
variables = X.columns
model = sm.OLS(y_train,X_train[variables]).fit()
stat_sig_vars = variables
while any(model.pvalues > .05):
    stat_sig_vars = stat_sig_vars[model.pvalues <= .05]
    model = sm.OLS(y_train,X_train[stat_sig_vars]).fit()

# report statistics on test set
model = sm.OLS(y_test,X_test[stat_sig_vars]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.954
Model:                            OLS   Adj. R-squared:                  0.952
Method:                 Least Squares   F-statistic:                     632.3
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          7.30e-159
Time:                        16:12:09   Log-Likelihood:                -770.64
No. Observations:                 253   AIC:                             1557.
Df Residuals:                     245   BIC:                             1586.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0040      0.052     -0.077      0.939      -0.107       0.099
ZN             0.0209      0.022      0.947      0.345      -0.023       0.064
CHAS           2.0500      1.319      1.554      0.121      -0.548       4.648
RM             5.2624      0.376     14.012      0.000       4.523       6.002
DIS           -0.5661      0.241     -2.354      0.019      -1.040      -0.092
PTRATIO       -0.4688      0.152     -3.092      0.002      -0.767      -0.170
B              0.0164      0.004      4.569      0.000       0.009       0.023
LSTAT         -0.4770      0.060     -7.914      0.000      -0.596      -0.358
==============================================================================
Omnibus:                       97.592   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              488.778
Skew:                           1.476   Prob(JB):                    7.30e-107
Kurtosis:                       9.136   Cond. No.                     1.47e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Model selection by minimizing the AIC

In [18]:
def minAIC(X,y):
    variables = X.columns
    model = sm.OLS(y,X[variables]).fit()
    while True:
        maxp = np.max(model.pvalues)
        newvariables = variables[model.pvalues < maxp]
        newmodel = sm.OLS(y,X[newvariables]).fit()
        if newmodel.aic < model.aic:
            model = newmodel
            variables = newvariables
        else:
            break
    return model,variables

model,variables = minAIC(X,y)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                     1162.
Date:                Fri, 17 Apr 2020   Prob (F-statistic):               0.00
Time:                        16:12:09   Log-Likelihood:                -1524.6
No. Observations:                 506   AIC:                             3069.
Df Residuals:                     496   BIC:                             3111.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0898      0.034     -2.630      0.009      -0.157      -0.023
ZN             0.0512      0.014      3.630      0.000       0.024       0.079
CHAS           2.7212      0.892      3.052      0.002       0.970       4.473
RM             5.7113      0.245     23.353      0.000       5.231       6.192
DIS           -0.8664      0.167     -5.185      0.000      -1.195      -0.538
RAD            0.1820      0.063      2.867      0.004       0.057       0.307
TAX           -0.0109      0.003     -3.292      0.001      -0.017      -0.004
PTRATIO       -0.4004      0.109     -3.682      0.000      -0.614      -0.187
B              0.0146      0.003      5.475      0.000       0.009       0.020
LSTAT         -0.4499      0.042    -10.596      0.000      -0.533      -0.366
==============================================================================
Omnibus:                      198.034   Durbin-Watson:                   0.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1249.000
Skew:                           1.575   Prob(JB):                    6.07e-272
Kurtosis:                      10.022   Cond. No.                     2.24e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.24e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
# select on training set, fit on test set 
model,variables = minAIC(X_train, y_train)
model = sm.OLS(y_test,X_test[variables]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.955
Model:                            OLS   Adj. R-squared:                  0.954
Method:                 Least Squares   F-statistic:                     521.3
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          5.42e-158
Time:                        16:12:09   Log-Likelihood:                -766.00
No. Observations:                 253   AIC:                             1552.
Df Residuals:                     243   BIC:                             1587.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.0085      0.057     -0.149      0.882      -0.120       0.103
ZN             0.0374      0.023      1.654      0.100      -0.007       0.082
CHAS           1.9031      1.303      1.461      0.145      -0.663       4.469
RM             5.4851      0.378     14.527      0.000       4.741       6.229
DIS           -0.7162      0.250     -2.862      0.005      -1.209      -0.223
RAD            0.2473      0.099      2.503      0.013       0.053       0.442
TAX           -0.0159      0.005     -3.010      0.003      -0.026      -0.006
PTRATIO       -0.3330      0.165     -2.022      0.044      -0.657      -0.009
B              0.0167      0.004      4.585      0.000       0.010       0.024
LSTAT         -0.4297      0.062     -6.961      0.000      -0.551      -0.308
==============================================================================
Omnibus:                      103.489   Durbin-Watson:                   2.065
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              592.224
Skew:                           1.529   Prob(JB):                    2.51e-129
Kurtosis:                       9.843   Cond. No.                     2.26e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [20]:
# compare with p values on training set 
model = sm.OLS(y_train,X_train[variables]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.964
Model:                            OLS   Adj. R-squared:                  0.963
Method:                 Least Squares   F-statistic:                     654.2
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          1.78e-169
Time:                        16:12:09   Log-Likelihood:                -751.95
No. Observations:                 253   AIC:                             1524.
Df Residuals:                     243   BIC:                             1559.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CRIM          -0.1356      0.043     -3.174      0.002      -0.220      -0.051
ZN             0.0584      0.018      3.228      0.001       0.023       0.094
CHAS           3.9211      1.230      3.187      0.002       1.497       6.345
RM             5.9295      0.328     18.096      0.000       5.284       6.575
DIS           -0.9108      0.231     -3.936      0.000      -1.367      -0.455
RAD            0.1159      0.083      1.397      0.164      -0.047       0.279
TAX           -0.0060      0.004     -1.400      0.163      -0.014       0.002
PTRATIO       -0.5235      0.149     -3.514      0.001      -0.817      -0.230
B              0.0143      0.004      3.346      0.001       0.006       0.023
LSTAT         -0.4630      0.058     -7.931      0.000      -0.578      -0.348
==============================================================================
Omnibus:                      121.312   Durbin-Watson:                   2.062
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              780.111
Skew:                           1.810   Prob(JB):                    3.99e-170
Kurtosis:                      10.804   Cond. No.                     2.26e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.26e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Another method for model selection: LASSO CV

In [21]:
# We use the base estimator LassoCV since the L1 norm promotes sparsity of features.
clf = LassoCV()
variables = X.columns

# Set a minimum threshold of .5 (controls number of features selected)
sfm = SelectFromModel(clf, threshold=.5)
sfm.fit(X_train, y_train)
selected_features = variables[sfm.get_support()]
selected_features

Index(['RM', 'DIS', 'PTRATIO', 'LSTAT'], dtype='object')

In [22]:
# fit model on test set, using just selected features, to report statistics
model = sm.OLS(y_test,X_test[selected_features]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.948
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     1143.
Date:                Fri, 17 Apr 2020   Prob (F-statistic):          6.81e-159
Time:                        16:12:10   Log-Likelihood:                -784.70
No. Observations:                 253   AIC:                             1577.
Df Residuals:                     249   BIC:                             1592.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
RM             5.9550      0.357     16.688      0.000       5.252       6.658
DIS           -0.2204      0.175     -1.262      0.208      -0.564       0.124
PTRATIO       -0.4275      0.139     -3.078      0.002      -0.701      -0.154
LSTAT         -0.5102      0.061     -8.341      0.000      -0.631      -0.390
==============================================================================
Omnibus:                       87.742   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              405.005
Skew:                           1.333   Prob(JB):                     1.13e-88
Kurtosis:                       8.595   Cond. No.                         27.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Random real features

In [23]:
# random birthdays (not randomizing year)
def random_date_generator(start_date='1925-01-01', range_in_days=365):
    random_date = np.datetime64(start_date) + int(np.random.rand()*range_in_days) + int(365*95*np.random.rand())
    return random_date

random_date = random_date_generator()
random_date

numpy.datetime64('2016-03-24')